# 실행전 준비(git clone)

데이터 출처: https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=580

Colab이 아닌 로컬 아나콘다에서 실행시, HuggingFaceEmbeddings를 이용한 

embedding model이 다운되지 않는 현상 발생

따라서 모델을 git clone으로 직접 다운로드 후 실행

In [ ]:
!git clone https://huggingface.co/jhgan/ko-sroberta-multitask models/ko-sroberta-multitask

# 라이브러리 import

In [1]:
# 데이터 출처: https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=580

In [ ]:
import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

import numpy as np
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
#api key 가져오기
#로컬 아나콘다에서 실행하여 userdata.get이 아닌 load_dotenv 사용
from dotenv import load_dotenv

load_dotenv(verbose=True)

HF_TOKEN = os.getenv('HF_TOKEN')

In [ ]:
#huggingface 로그인
from huggingface_hub import login
login(HF_TOKEN, add_to_git_credential=True)

In [ ]:
#vectorstore 저장할 경로 지정
db_name = "vector_db"

In [ ]:
from langchain_core.documents import Document
from pathlib import Path
import json
root = Path("판결문 데이터 위치")
folders = root.rglob("*.json")

docs = []

for folder in folders:  
    with open(folder,'r',  encoding = 'UTF-8') as f:
        case = json.load(f)
        
    info = case.get("info", {})
    concerned = case.get("concerned", {})
    disposal = case.get("disposal", {})
    assrs = case.get("assrs", {})
    disposal = case.get("disposal", {})
    facts = case.get("facts", {})
    dcss = case.get("dcss", {})
    close = case.get("close", {})

    # 공통 metadata
    metadata = {
        "case_number": info.get("caseNo"),
        "case_name": info.get("caseNm"),
        "court_name": info.get("courtNm"),
        "decision_date": info.get("judmnAdjuDe"),
        "case_field": info.get("caseField"),
        "relateLaword": info.get("relateLaword"),
    }

    acusrAssrs = "원고의 주장 " + "\n".join(assrs.get("acusrAssrs", []))
    if acusrAssrs.strip():
        docs.append(Document(
            page_content=acusrAssrs,
            metadata={**metadata, "section": "원고의 주장"}
        ))

    dedatAssrs = "피고의 주장 " + "\n".join(assrs.get("dedatAssrs", []))
    if dedatAssrs.strip():
        docs.append(Document(
            page_content=dedatAssrs,
            metadata={**metadata, "section": "피고의 주장"}
        ))

    # 사실관계 부분 합치기
    facts_text = "사실관계 " + "\n".join(facts.get("bsisFacts", []))
    if facts_text.strip():
        docs.append(Document(
            page_content=facts_text,
            metadata={**metadata, "section": "사실관계"}
        ))

    # 법원의 판단 부분 합치기
    dcss_text = "법원의 판단 " + "\n".join(dcss.get("courtDcss", []))
    if dcss_text.strip():
        docs.append(Document(
            page_content=dcss_text,
            metadata={**metadata, "section": "법원의 판단"}
        ))

    # 주문 부분
    disposal_text = "주문 " + "\n".join(disposal.get("disposalcontent", []))
    if disposal_text.strip():
        docs.append(Document(
            page_content=disposal_text,
            metadata={**metadata, "section": "주문"}
        ))

    # 결론 부분
    close_text = "결론 " + "\n".join(close.get("cnclsns", []))
    if close_text.strip():
        docs.append(Document(
            page_content=close_text,
            metadata={**metadata, "section": "결론"}
        ))

In [ ]:
print(docs[0])

In [ ]:
len(docs)

In [ ]:
#CharacterTextSplitter로 chunk 나누기
text_splitter = CharacterTextSplitter(chunk_size=1200, chunk_overlap=200)
chunks = text_splitter.split_documents(docs)

In [ ]:
for chunk in chunks:
    if chunk.metadata:
        for key, value in chunk.metadata.items():
            if isinstance(value, list):
                # 리스트를 쉼표로 구분된 문자열로 변환
                chunk.metadata[key] = ', '.join(map(str, value))
            elif isinstance(value, dict):
                # 딕셔너리를 JSON 문자열로 변환
                import json
                chunk.metadata[key] = json.dumps(value, ensure_ascii=False)

In [ ]:
print(f"Total number of chunks: {len(chunks)}")
print(f"Document types found: {set(doc.metadata['case_name'] for doc in docs)}")

In [ ]:
#임베딩 모델 파일 다운이 안되어서 git clone으로 직접 다운로드
#!git clone https://huggingface.co/jhgan/ko-sroberta-multitask

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="./models/ko-sroberta-multitask"  # 로컬 모델 경로
)
                                                 
# embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
# 로컬 모델이 아닌 huggingface 모델 지정시 자동 다운로드

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
# embeddings = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

if os.path.exists(db_name):
    Chroma(persist_directory=db_name, embedding_function=embeddings).delete_collection()

vectorstore = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_name)
print(f"Vectorstore created with {vectorstore._collection.count()} documents")

In [ ]:
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

# vectorstore 테스트

강의에서 배포된 코드를 기반으로 수정

RunnableWithMessageHistory와 같은 몇몇 부분은 강의 라이브러리 버전과 현재 사용중인 부분이맞지 않아서, 

완전히 갈아엎음

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# 환경 변수 설정

openrouter_url = "https://openrouter.ai/api/v1"

# LLM 설정
llm = ChatOpenAI(
    temperature=0.7, #숫자가 올라갈수록 ai의 답변 창의성 증가, 용도에 맞게 수치 설정
    model_name=MODEL,
    base_url=openrouter_url,
    api_key=os.environ['OPENROUTER_API_KEY']
)

# Retriever 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 문서 포맷팅 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 프롬프트 설정
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 질문에 답변하는 AI 어시스턴트입니다. 주어진 문서를 기반으로 답변하세요."),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "문서:\n{context}\n\n질문: {question}")
])

# 체인 구성
def get_context(input_dict):
    question = input_dict["question"]
    docs = retriever.invoke(question)
    return format_docs(docs)

chain = (
    RunnablePassthrough.assign(context=get_context)
    | prompt
    | llm
    | StrOutputParser()
)

# 세션별 메모리 저장소
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 대화 히스토리를 포함한 체인
conversation_chain = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history",
)

# 사용 예시
session_id = "user123"
response = conversation_chain.invoke(
    {"question": "안녕하세요?"},  # 딕셔너리로 전달
    config={"configurable": {"session_id": session_id}}
)
print(response)

# 대화 계속
response2 = conversation_chain.invoke(
    {"question": "이 문서에 대해 설명해주세요."},
    config={"configurable": {"session_id": session_id}}
)
print(response2)

In [ ]:
def chat(question, history):
    response = conversation_chain.invoke(
    {"question": question},  # 딕셔너리로 전달
    config={"configurable": {"session_id": session_id}}
)
    return response

In [ ]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True, share = True)